<h2>Segmenting and Grouping Neighborhood Vulnerable to Covid-19 Pandemic</h2>

<h2> By Michael Kumakech</h2>

<h2>PART  1: Model Development </h2>

In this section, we will develop several models that will predict the number deaths due to Convid-19 Pandemic in the World using the variables or features. This is just an estimate but should give us an objective idea of how much the Pandemic will cost.

Data Analytics, we often use Model Development to help us predict future observations from the data we have.
In  Artificial Intelligence, a Model will help us understand the exact relationship between different variables and how these variables are used to predict the result.


<b> Import Libaries</b>

In [1]:
import requests
import lxml.html as lh
import pandas as pd

<b> Remove the website to be put in the notebook</b>

In [2]:
WHO_url = 'https://www.ecdc.europa.eu/en/geographical-distribution-2019-ncov-cases' #assign the wiki page
#WHO_url = 'https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases'

page = requests.get(WHO_url) # create a handle to for contents of the wiki page

doc = lh.fromstring(page.content) # store content of the wiki page under doc

tr_elements = doc.xpath('//tr') # parse data stored between tr in the html

[len(T) for T in tr_elements[:12]] # check the length of the first 12 rows

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

<b> Check the table headers</b>

In [3]:
tr_elements = doc.xpath('//tr') # parse first row as header

col = [] # create empty list
i = 0

for t in tr_elements[0]: # for each row, store each first element (header) and an empty list
    i+=1
    name=t.text_content()
    print("%d:%s" % (i,name))
    col.append((name,[]))

1:Region
2:Places reporting cases
3:Cases
4:Deaths
5:Confirmed cases in the last 14 days


<b> Check the data in other Row</b>

In [4]:
for j in range(1,len(tr_elements)): # Because header is the first row, data would be store in the subsequent rows.
    T = tr_elements[j] #T is j'th row
    
    if len(T)!=5: #if row is not size 3, //tr data is not from the table.
        break
        
    i = 0 #i is the index of the first column
    
    for t in T.iterchildren(): #iterate through each element of the row
        data=t.text_content()
            
        col[i][1].append(data) #append the data to the empty list of the i'th column
            
        i+=1 #increment i for the next column

<b> What about the numbers of rows and columns</b>

In [5]:
[len(C) for (title,C) in col]

[207, 207, 207, 207, 207]

<b> Displays the data frame with three columns.</b>

In [12]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

In [7]:
df.head(12)

,Region,Places reporting cases,Cases,Deaths,Confirmed cases in the last 14 days
0,Africa,Algeria,1825,275,1416
1,Africa,Angola,19,2,15
2,Africa,Benin,35,1,29
3,Africa,Botswana,13,1,13
4,Africa,Burkina_Faso,484,27,304
5,Africa,Burundi,5,0,5
6,Africa,Cameroon,820,11,721
7,Africa,Cape_Verde,8,1,3
8,Africa,Central_African_Republic,11,0,5
9,Africa,Chad,11,0,6


In [13]:
df.tail(10)

,Region,Places reporting cases,Cases,Deaths,Confirmed cases in the last 14 days
197,Oceania,Australia,6289,57,2480
198,Oceania,Fiji,16,0,11
199,Oceania,French_Polynesia,51,0,17
200,Oceania,Guam,133,5,78
201,Oceania,New_Caledonia,18,0,3
202,Oceania,New_Zealand,1049,4,573
203,Oceania,Northern_Mariana_Islands,11,2,11
204,Oceania,Papua_New_Guinea,2,0,1
205,Other,Cases_on_an_international_conveyance_Japan,696,7,0
206,Total,,1734913,108306,1078045


<b> Checking the shapes</b>

In [9]:
df.shape

(207, 5)

<h2> Clean the dataframe</h2>

In [14]:
import pandas as pd
import numpy as np

<b> Remove Row 204 that have total  record</b>

In [15]:
# Delete row at index position Region: Total
df2 = df.drop([df.index[206]])
df2

,Region,Places reporting cases,Cases,Deaths,Confirmed cases in the last 14 days
0,Africa,Algeria,1825,275,1416
1,Africa,Angola,19,2,15
2,Africa,Benin,35,1,29
3,Africa,Botswana,13,1,13
4,Africa,Burkina_Faso,484,27,304
5,Africa,Burundi,5,0,5
6,Africa,Cameroon,820,11,721
7,Africa,Cape_Verde,8,1,3
8,Africa,Central_African_Republic,11,0,5
9,Africa,Chad,11,0,6


<h2> Data Analysis</h2>

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
%matplotlib inline

In [17]:
%%capture
! pip install seaborn

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [25]:
# Engine size as potential predictor variable of price
#sns.regplot(x="Cases", y="Deaths", data=df2)
#plt.ylim(0, )

<h2>Explore the Data</h2>

In [19]:
df2.describe()

,Region,Places reporting cases,Cases,Deaths,Confirmed cases in the last 14 days
count,206,206,206,206,206
unique,6,206,170,77,160
top,Europe,China,9,0,5
freq,54,1,4,45,8


<b>We can see we have missing values for the columns. </b>

In [20]:
print("number of NaN values for the column Region :", df2['Region'].isnull().sum())
print("number of NaN values for the column Places reporting cases:", df2['Places reporting cases'].isnull().sum())
print("number of NaN values for the column Cases :", df2['Cases'].isnull().sum())
print("number of NaN values for the column Deaths :", df2['Deaths'].isnull().sum())
#print("number of NaN values for the column Confirmed cases in the last 15 days :", df2['Confirmed cases in the last 15 days'].isnull().sum())

number of NaN values for the column Region : 0
number of NaN values for the column Places reporting cases: 0
number of NaN values for the column Cases : 0
number of NaN values for the column Deaths : 0


<h2>Exploratory data analysis</h2>

In [21]:
df2['Region'].value_counts().to_frame()

,Region
Europe,54
Africa,52
America,49
Asia,42
Oceania,8
Other,1


<h2> Model Development</h2>

In [22]:
#Import libraries 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression 
%matplotlib inline

<b>Import packages</b>

In [47]:
!conda install -c anaconda xlrd --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    xlrd-1.2.0                 |             py_0         108 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    ca-certificates: 2020.1.1-0        --> 2020.1.1-0        anaconda
    certifi:         2019.11.28-py36_0 --> 2020.4.5.1-py36_0 anaconda
    openssl:         1.1.1e-h7b6447c_0 --> 1.1.1-h7b6447c_0  anaconda
    xlrd:            1.2.0-py_0       

<h2>Model 1a: Simple Linear Regression Development</h2>


(a)We can Fit a linear regression model using the<B> Cases</B> feature and caculate the R^2 for <b>Deaths</b>

In [23]:
X = df2[['Cases']] 
Y = df2['Deaths'] 
lm = LinearRegression() 
lm 
lm.fit(X,Y)
lm.score(X, Y)

0.7205027189687778

<b>How could identified cases help us predict deaths? </b>

In [24]:
X = df2[['Cases']] 
Y = df2['Deaths'] 
lm = LinearRegression()
lm
lm.fit(X,Y)

Yhat=lm.predict(X)
Yhat[0:5]

array([183.14822279,  89.35398131,  90.18493804,  89.04237253,
       113.50366142])

<b> What is the value of the intercept ? </b>

In [25]:
lm.intercept_

88.36722018621413

<b>What is the value of the Slope ? </b>

In [26]:
lm.coef_

array([0.0519348])


<h2>What is the final estimated linear model we get?</h2>

As we saw above, we should get a final linear model with the structure: <I>  Yhat = a + b X </I>

Plugging in the actual values we get: 

<b>Deaths = 88.367 + 0.0519 x cases </b> as of data update of 12th April 2020 at 12:56pm. However, 
 this Linear model keeps changing as the numbers of cases and deaths increases 

<h2>Model 1b: Simple Linear Regression</h2>

 (b)We can Fit a linear regression model using the <b>Confirmed cases in the last 14 days </b>feature and caculate the R^2 for <b>Deaths</b>

In [47]:
X = df2[['Confirmed cases in the last 14 days']] 
Y = df2['Deaths'] 
lm = LinearRegression() 
lm 
lm.fit(X,Y)
lm.score(X, Y)

0.5986593289391934

When you compare Model 1a: Simple Linear Regression and Model 1b: Simple Linear Regression the R^2 values are ~<b>72%</b> and ~<b>60% </B>. Thus, Model 1 (a) performs better than that in (b). We use Cases instead of Confirmed cases in the last 14 days when developing Linear Regression model.

<h2> Measures for In-Sample Evaluation  of  Linear Regression</h2>

<h2>R-squared </h2> R squared, also known as the coefficient of determination, is a measure to indicate how close the data is to the fitted regression line. The value of the R-squared is the percentage of variation of the response variable (y) that is explained by a linear model.
<h2>Mean Squared Error (MSE)</h2>
The Mean Squared Error measures the average of the squares of errors, that is, the difference between actual value (y) and the estimated value (ŷ).


<h2>Model 1: Simple Linear Regression</h2>

In [27]:
#Cases_fit
lm.fit(X, Y)
# Find the R^2
print('The R-square is: ', lm.score(X, Y))


The R-square is:  0.7205027189687778


We can say that ~ <b>72%</b> of the variation of the Deaths is explained by this simple linear model "Cases_fit".

<h2>Let's calculate the MSE</h2> We can predict the output i.e., "yhat" using the predict method, where X is the input variable:

In [28]:
Yhat=lm.predict(X)
print('The output of the first four predicted value is: ', Yhat[0:4])


The output of the first four predicted value is:  [183.14822279  89.35398131  90.18493804  89.04237253]


lets import the function <b>mean_squared_error</b> from the <b>module metrics</b>

In [29]:
from sklearn.metrics import mean_squared_error

<b> Wwe compare the predicted results with the actual results</b>

In [31]:
mse = mean_squared_error(df2['Deaths'], Yhat)
print('The mean square error of price and predicted value is: ', mse)

The mean square error of price and predicted value is:  1851204.6582469824


<h2> Multiple Linear Regression</h2>

<B> 

What if we want to predict car price using more than one variable?</B>

If we want to use more variables in our model to predict Deaths, we can use <b>Multiple Linear Regression</b>. Multiple Linear Regression is very similar to Simple Linear Regression, but this method is used to explain the relationship between one continuous response (dependent) variable and two or more predictor (independent) variables. Most of the real-world regression models involve multiple predictors. We will illustrate the structure by using four predictor variables, but these results can generalize to any integer:

<b> Let's develop a model using these variables as the predictor variables. </b>

In [32]:
Z = df2[['Cases', 'Confirmed cases in the last 14 days']]

<b>Fit the linear model using the four above-mentioned variables.</b>

In [33]:
lm.fit(Z, df2['Deaths'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

<b> What is the value of the intercept(a)?</b>

In [34]:
lm.intercept_

10.921256839909915

<b> What are the values of the coefficients (b1, b2, b3, b4)?</b>

In [35]:
lm.coef_

array([ 0.1326288, -0.1150631])

<b>What is the final estimated linear model that we get?</b>

As we saw above, we should get a final linear function with the structure

<h2> Deaths = 10.921 + 0.133*Cases - 0.115* Confirmed cases in the last 14 days</h2> As at 1:15pm  12th April 2020. This model keeps on changing as data is being updated daily.

<h2>Measures for In-Sample Evaluation of Multiple Linear Regression</h2>

<h2>Model 2: Multiple Linear Regression</h2>

<b>Let's calculate the R^2</b>

In [37]:
# fit the model 
lm.fit(Z, df2['Deaths'])
# Find the R^2
print('The R-square is: ', lm.score(Z, df2['Deaths']))


The R-square is:  0.7972667695344539


We can say that ~ <b>79.73 %</b> of the variation of price is explained by this multiple linear regression "multi_fit".

<b>Let's calculate the MSE</b>

We produce a prediction

In [38]:
Y_predict_multifit = lm.predict(Z)

we compare the predicted results with the actual results

In [39]:
print('The mean square error of price and predicted value using multifit is: ', \
      mean_squared_error(df2['Deaths'], Y_predict_multifit))


The mean square error of price and predicted value using multifit is:  1342770.488623659


<h2>Decision Making: Determining a Good Model Fit</h2>Now that we have visualized the different models, and generated the R-squared and MSE values for the fits, how do we determine a good model fit? 


<h2>What is a good R-squared value?</h2>


When comparing models, the model with the higher R-squared value is a better fit for the data. 

<h2>What is a good MSE?:</h2>
When comparing models, the model with the smallest MSE value is a better fit for the data.


<b>Let's take a look at the values for the different models.</b>
<b><I>Simple Linear Regression:</I></B> Using Cases as a Predictor Variable of Deaths. 
•	R-squared: 0.7205027189687778
•	MSE: 1.85 x10^6



<b><I>Multiple Linear Regression:</b></I> Using Cases, and Confirmed cases in the last 14 days as Predictor Variables of Deaths. 
•	R-squared: 0.7972667695344539
•	MSE: 1.34 x10^6


<h2>Conclusion:</h2>

Comparing these three models, we conclude that the <b>Multiple Linear Regression model</b> is the best model to be able to predict Deaths from our dataset. This result makes sense, since we have 3 variables in total, and we know that more than one of those variables are potential predictors of the final numbers of Deaths.

Model for predicting deaths based on cases and Confirmed cases in the last 14 days for PART 1 of our project is <h2> Deaths = 10.921 + 0.133*Cases - 0.115* Confirmed cases in the last 14 days</h2> As at 1:15pm  12th April 2020. This model keeps on changing as data is being updated daily.

<h2>Thank you for completing this notebook</h2>

<h2>About the Author:</h2>

This notebook was written by <a>Michael Kumakech</a>.
Michael Kumakech is Lecturer at Faculty of Science and Technology Cavendish University in Uganda, and holds a Professional Certificate in IBM Data Science, a  MSc in Data Communication and Software Engineering. His research focused on using Machine Learning, Signal Processing, and Computer Vision to determine how videos impact human cognition. Michael has been working for Cavendish University in Uganda since 2019.


<h2> PART 2: Machine Learning</h2> This is the Next......for week of 19th April 2020 to 25th April 2020. Class of Artificial Intelligence must participate actively in running the codes. 